In [1]:
"""
Train on images split into directories. This assumes we've split
our videos into frames and moved them to their respective folders.
Use keras 2+ and tensorflow 1+
Based on:
https://keras.io/preprocessing/image/
and
https://keras.io/applications/
"""
import os
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.optimizers import SGD, Adagrad, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Reshape, Activation, Multiply, GlobalMaxPooling2D
from keras.layers import LSTM, TimeDistributed, Dropout, Input, Flatten, Lambda, Concatenate, Average
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
#from utils.clr.clr_callback import * 
#from UCFdata import DataSet

from keras.layers import concatenate
from keras.regularizers import l1, l2



Using TensorFlow backend.


In [2]:
from keras.layers import Dense, Wrapper 
import keras.backend as K 


class DropConnect(Wrapper): 
    def __init__(self, layer, prob=1., **kwargs): 
        self.prob = prob 
        self.layer = layer 
        super(DropConnect, self).__init__(layer, **kwargs) 
        if 0. < self.prob < 1.: 
            self.uses_learning_phase = True 

 
    def build(self, input_shape): 
        if not self.layer.built: 
            self.layer.build(input_shape) 
            self.layer.built = True 
        super(DropConnect, self).build() 

 
    def compute_output_shape(self, input_shape): 
        return self.layer.compute_output_shape(input_shape) 

 
    def call(self, x): 
        if 0. < self.prob < 1.: 
            self.layer.kernel = K.in_train_phase(K.dropout(self.layer.kernel, self.prob), self.layer.kernel) 
            self.layer.bias = K.in_train_phase(K.dropout(self.layer.bias, self.prob), self.layer.bias) 
        return self.layer.call(x) 


# CODE FOR INCEPTIONV3

In [3]:
import numpy as np
import json

from keras.utils.data_utils import get_file
from keras import backend as K

CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'


def preprocess_input(x, dim_ordering='default'):
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}

    if dim_ordering == 'th':
        x[:, 0, :, :] -= 103.939
        x[:, 1, :, :] -= 116.779
        x[:, 2, :, :] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, ::-1, :, :]
    else:
        x[:, :, :, 0] -= 103.939
        x[:, :, :, 1] -= 116.779
        x[:, :, :, 2] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, :, :, ::-1]
    return x


def decode_predictions(preds, top=5):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 1000)). '
                         'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        results.append(result)
    return results

In [4]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten, Dense, Input, BatchNormalization, merge
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.preprocessing import image
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K


TH_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_th_dim_ordering_th_kernels.h5'
TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_tf_dim_ordering_tf_kernels.h5'
TH_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_th_dim_ordering_th_kernels_notop.h5'
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


def conv2d_bn(x, nb_filter, nb_row, nb_col,
              border_mode='same', subsample=(1, 1),
              name=None):
    '''Utility function to apply conv + BN.
    '''
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_dim_ordering() == 'th':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Convolution2D(nb_filter, nb_row, nb_col,
                      subsample=subsample,
                      activation='relu',
                      border_mode=border_mode,
                      name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name)(x)
    return x


def InceptionV3(include_top=True, weights='imagenet',
                input_tensor=None):
    '''Instantiate the Inception v3 architecture,
    optionally loading weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_dim_ordering="tf"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The dimension ordering
    convention used by the model is the one
    specified in your Keras config file.
    Note that the default input image size for this model is 299x299.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
    # Returns
        A Keras model instance.
    '''
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')
    # Determine proper input shape
    if K.image_dim_ordering() == 'th':
        if include_top:
            input_shape = (3, 299, 299)
        else:
            input_shape = (3, None, None)
    else:
        if include_top:
            input_shape = (299, 299, 3)
        else:
            input_shape = (None, None, 3)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor)
        else:
            img_input = input_tensor

    if K.image_dim_ordering() == 'th':
        channel_axis = 1
    else:
        channel_axis = 3

    x = conv2d_bn(img_input, 32, 3, 3, subsample=(2, 2), border_mode='valid')
    x = conv2d_bn(x, 32, 3, 3, border_mode='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 80, 1, 1, border_mode='valid')
    x = conv2d_bn(x, 192, 3, 3, border_mode='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    for i in range(3):
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
        x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool])

    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, subsample=(2, 2), border_mode='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3,
                             subsample=(2, 2), border_mode='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch3x3dbl, branch_pool])

    # mixed 4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool])

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 160, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool])

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 192, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 160, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool])

    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                          subsample=(2, 2), border_mode='valid')

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3,
                            subsample=(2, 2), border_mode='valid')

    branch_pool = AveragePooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch7x7x3, branch_pool])

    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = concatenate([branch3x3_1, branch3x3_2])

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2])

        branch_pool = AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool])

    if include_top:
        # Classification block
        x = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
        x = Flatten(name='flatten')(x)
        x = Dense(1000, activation='softmax', name='predictions')(x)

    # Create model
    model = Model(img_input, x)

    # load weights
    if weights == 'imagenet':
        if K.image_dim_ordering() == 'th':
            if include_top:
                weights_path = get_file('inception_v3_weights_th_dim_ordering_th_kernels.h5',
                                        TH_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='b3baf3070cc4bf476d43a2ea61b0ca5f')
            else:
                weights_path = get_file('inception_v3_weights_th_dim_ordering_th_kernels_notop.h5',
                                        TH_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='79aaa90ab4372b4593ba3df64e142f05')
            model.load_weights(weights_path)
            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image dimension ordering convention '
                              '(`image_dim_ordering="th"`). '
                              'For best performance, set '
                              '`image_dim_ordering="tf"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
                convert_all_kernels_in_model(model)
        else:
            if include_top:
                weights_path = get_file('inception_v3_weights_tf_dim_ordering_tf_kernels.h5',
                                        TF_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='fe114b3ff2ea4bf891e9353d1bbfb32f')
            else:
                weights_path = get_file('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                        TF_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='2f3609166de1d967d1a481094754f691')
            model.load_weights(weights_path)
            if K.backend() == 'theano':
                convert_all_kernels_in_model(model)
    return model


def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

# END OF CODE FOR INCEPTIONV3

In [5]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


In [6]:
# def get_generators():
#     datagen = ImageDataGenerator(rescale=1. / 255)
#     train_generator = datagen.flow_from_directory(
#           'data/train',
#           target_size=(224, 224),
#           batch_size=batch_size,
#           class_mode='categorical',  # this means our generator will only yield batches of data, no labels
#           shuffle=True,
#           classes=['angry','happy', 'sad', 'submissive'])

#     validation_generator = datagen.flow_from_directory(
#           'data/val',
#           target_size=(224, 224),
#           batch_size=batch_size,
#           class_mode='categorical',  # this means our generator will only yield batches of data, no labels
#           shuffle=True,
#           classes=['angry','happy','sad', 'submissive'])
#     return train_generator, validation_generator


In [7]:
def generate_generator_multiple(generator,dir1, dir2, dir3, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          classes=['angry','happy','sad', 'submissive'],
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          classes=['angry','happy','sad', 'submissive'],
                                          seed=7)
    
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          classes=['angry','happy','sad', 'submissive'],
                                          seed=7)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            yield [X1i[0], X2i[0], X3i[0]], X2i[1]  #Yield both images and their mutual label

In [8]:
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.models import Model
import os


def CNN(input_shape, classes, weights_dir, include_top=True):
    '''
    The CNN for optical flow input.
    Since optical flow is not a common image, we cannot finetune pre-trained ResNet (The weights trained on imagenet is
    for images and thus is meaningless for optical flow)
    :param input_shape: the shape of optical flow input
    :param classes: number of classes
    :return:
    '''
    optical_flow_input = Input(shape=input_shape)

    x = Convolution2D(96, kernel_size=(7, 7), strides=(2, 2), padding='same', name='tmp_conv1')(optical_flow_input)
    x = DropConnect(Dense(96, activation='relu'), prob=0.5)(x)
    x = BatchNormalization(axis=3)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Convolution2D(256, kernel_size=(5, 5), strides=(2, 2), padding='same', name='tmp_conv2')(x)
    x = DropConnect(Dense(256, activation='relu'), prob=0.5)(x)
    x = BatchNormalization(axis=3)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', name='tmp_conv3')(x)
    x = DropConnect(Dense(512, activation='relu'), prob=0.5)(x)
    x = BatchNormalization(axis=3)(x)

    x = Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', name='tmp_conv4')(x)
    x = DropConnect(Dense(512, activation='relu'), prob=0.5)(x)
    x = BatchNormalization(axis=3)(x)

    x = Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', name='tmp_conv5')(x)
    x = DropConnect(Dense(512, activation='relu'), prob=0.5)(x)
    x = BatchNormalization(axis=3)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

#     x = Flatten()(x)
#     x = Dense(4096, activation='relu', name='tmp_fc6')(x)
#     x = Dropout(0.9)(x)

#     x = Dense(2048, activation='relu', name='tmp_fc7')(x)
#     x = Dropout(0.9)(x)

    if include_top:
        x = Dense(classes, activation='softmax', name='tmp_fc101')(x)

    
    
    model = Model(inputs=optical_flow_input, outputs=x, name='temporal_CNN')

    
    if weights_dir == 'imagenet':
        if K.image_dim_ordering() == 'th':
            if include_top:
                weights_path = get_file('inception_v3_weights_th_dim_ordering_th_kernels.h5',
                                        TH_WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='b3baf3070cc4bf476d43a2ea61b0ca5f')
            else:
                weights_path = get_file('inception_v3_weights_th_dim_ordering_th_kernels_notop.h5',
                                        TH_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models',
                                        md5_hash='79aaa90ab4372b4593ba3df64e142f05')
            model.load_weights(weights_path)
            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image dimension ordering convention '
                              '(`image_dim_ordering="th"`). '
                              'For best performance, set '
                              '`image_dim_ordering="tf"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
                convert_all_kernels_in_model(model)
    else:
        if os.path.exists(weights_dir):
            model.load_weights(weights_dir, by_name=True)

    return model

In [9]:
def get_model(weights='imagenet'):
    # create the base pre-trained model
    base_model_local = InceptionV3(weights=weights, include_top=False)
    x1 = base_model_local.output
    x1 = GlobalAveragePooling2D()(x1)
    x1 = Dense(256, activation='relu')(x1)
    x1 = DropConnect(Dense(256, activation='relu'), prob=0.9)(x1)
   
                     

    base_model_globallstm = InceptionV3(weights=weights, include_top=False)
    x2 = base_model_globallstm.output
    x2 = GlobalAveragePooling2D()(x2)
    x2 = Dense(256, activation='relu')(x2)
    x2 = DropConnect(Dense(256, activation='relu'), prob=0.9)(x2)

          
                                   
                                   
    base_model_optical = CNN((224,224,3),4,'', include_top=False)                              
    x3 = base_model_optical.output
    x3 = Convolution2D(2048, kernel_size=(3, 3), strides=(1, 1))(x3)
    x3 = Reshape((25,2048))(x3)
    x3 = LSTM(256,dropout=0.2,input_shape=(25,2048))(x3)
    x3 = DropConnect(Dense(256, activation='relu'), prob=0.9)(x3)

    
                                   
    x = Average(name='gadogado_layer')([x1, x2, x3])                               
    x = Dense(1024, activation='relu')(x)
    x = DropConnect(Dense(1024, activation='relu'), prob=0.9)(x)
#     x = Dropout(0.9)(x)
    
    predictions = Dense(4, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=[base_model_local.input,base_model_globallstm.input,base_model_optical.input], outputs=predictions)

    for layer in base_model_local.layers:
        layer.trainable = False
    for layer in base_model_globallstm.layers:
        layer.trainable = False
#     for layer in base_model_optical.layers:
#         layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=Adagrad(lr=0.0001, decay = 1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
img_height = 224
img_width = 224
train_data = 23750
val_data = 7100
batch_size = 25
# Helper: Save the min val_loss model in each epoch.

checkpointer = ModelCheckpoint(
    filepath='incept(global)_incept(local)_incept(of)_avg.{epoch:03d}-{val_loss:.2f}-{val_acc:.2f}.hdf5', 
    #save format is epoch-loss-acc-time-date(mmddyyyy)
    verbose=1,
    monitor='val_acc',
    save_best_only=True)

# Helper: Stop when we stop learning.
# patience: number of epochs with no improvement after which training will be stopped.
early_stopper = EarlyStopping(patience=10)

#Cyclic Learning Rate. base lr is the minimum lowest bound
#clr_triangular = CyclicLR(mode='triangular', base_lr = 0.00005)
# Helper: TensorBoard
tensorboard = TensorBoard(log_dir='utils/logs/')
traindatagen = ImageDataGenerator(rescale=1. / 255)
valdatagen = ImageDataGenerator(rescale=1. / 255)
train_generator=generate_generator_multiple(generator=traindatagen,
                                           dir1='data/train',
                                           dir2='object_data/train',
                                            dir3='OF_data/train',
                                            batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)       
     
validation_generator=generate_generator_multiple(valdatagen,
                                          dir1='data/val',
                                          dir2='object_data/val',
                                            dir3='OF_data/val',
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_width)     
model = get_model()
model.summary()
nb_epoch = 30
model = model.fit_generator(
    train_generator,
    steps_per_epoch=train_data/batch_size,
    validation_data=validation_generator,
    validation_steps=val_data/batch_size,
    epochs=nb_epoch,
    callbacks=[checkpointer])

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name=None, strides=(2, 2), padding="valid")`
C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name=None, strides=(1, 1), padding="valid")`
C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name=None, strides=(1, 1), padding="same")`
C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (1, 1), activation="relu", name=None, strides=(1, 1), padding="valid")`
C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Con

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 896         input_2[0][0]               

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
Found 7100 images belonging to 4 classes.
Found 7100 images belonging to 4 classes.
Found 23750 images belonging to 4 classes.
Found 7100 images belonging to 4 classes.
Found 23750 images belonging to 4 classes.
Found 23750 images belonging to 4 classes.
950/950 [==============================] - 1063s 1s/step - loss: 1.7289 - acc: 0.3371 - val_loss: 1.3882 - val_acc: 0.3183

Epoch 00001: val_acc improved from -inf to 0.31831, saving model to incept(global)_incept(local)_incept(of)_avg.001-1.39-0.32.hdf5
Epoch 2/30
950/950 [==============================] - 1036s 1s/step - loss: 1.5852 - acc: 0.2832 - val_loss: 1.3817 - val_acc: 0.3290

Epoch 00002: val_acc improved from 0.31831 to 0.32901, saving model to incept(global)_incept(local)_incept(of)_avg.002-1.38-0.33.hdf5
Epoch 3/30
557/950 [================>.............] - ETA: 6:15 - loss: 1.3458 - acc: 0.3698

# Testing Purpose

In [ ]:
from keras.models import load_model
model = load_model('utils/checkpoints/lstm/incept(global)_incept(local)_avg.011-2.28-0.34.hdf5')

In [ ]:
"""
Classify test images set through our CNN.
Use keras 2+ and tensorflow 1+
It takes a long time for hours.
"""
import numpy as np
import operator
import random
import glob
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator


# CNN model evaluate

test_data_gen = ImageDataGenerator(rescale=1. / 255)
batch_size = 25
test_path = 'data/test'  #path to your validation / test videos
test_generator = test_data_gen.flow_from_directory(test_path, target_size=(224, 224),
                                                   batch_size=batch_size, classes=['angry','happy','sad', 'submissive'],
                                                   class_mode='categorical',shuffle = False)

In [ ]:
test_generator=generate_generator_multiple(test_data_gen,
                                          dir1='data/test',
                                          dir2='object_data/test',
                                          batch_size=batch_size,
                                          img_height=224,
                                          img_width=224)  

In [ ]:
test_data_num = 14950          
#test_generator.reset() #to avoid having bugs in generator.
# if you dont invoke .reset(), it will starts to mix the order of the array from predicted generator

predicted_array = model.predict_generator(test_generator, steps = test_data_num/batch_size)
len(predicted_array) #check if generated data is the same as test_data_num

### Documentation section

In [ ]:
frame = [] #check all frame names for each validation / test videos
filenames = [] #check all filenames for each validation / test videos
gt = [] #ground truth storage for each validation / test videos
length = [] #how many frames belongs to which test / validation videos
final_predicted_array = [] #final predicted results for all videos

catfiles = os.listdir(test_path)
for i in range(len(catfiles)):
    subvideos = os.listdir(os.path.join(test_path,catfiles[i]))
    filenames.extend(subvideos)
    for j in range(len(subvideos)):
        contentvideos = os.listdir(os.path.join(test_path,catfiles[i],subvideos[j]))
        frame.append(contentvideos)
        length.append(len(contentvideos))
        gt.append(i)
cum_length = np.cumsum(length)
test2 = predicted_array[:cum_length[0]]
final_predicted_array.append(test2)
for i in range(len(cum_length)-1):
    test2 = predicted_array[cum_length[i]:cum_length[i+1]]
    final_predicted_array.append(test2)
votepred = []
meanpred = []

for i in range(len(final_predicted_array)):
    votectg = np.bincount(np.argmax(final_predicted_array[i], axis=1))
    votepred.append(np.argmax(votectg))
    meanctg = np.argmax(np.mean(final_predicted_array[i], axis=0))
    meanpred.append(meanctg)

vote = 0
mean = 0
for i in range(len(gt)):
    if(gt[i] == votepred[i]):
        vote += 1
    if(gt[i] == meanpred[i]):
        mean += 1
        
vote_acc = vote/len(gt)
mean_acc = mean/len(gt)

print("Voting accuracy : ", vote_acc)
print("Mean accuracy : ", mean_acc)